In [0]:
import tensorflow
import pandas as pd
import tensorflow.compat.v1 as tf
from sklearn.model_selection import train_test_split
tf.disable_v2_behavior()

In [56]:
base = pd.read_csv('/content/drive/My Drive/TensorFlow_1.x_guide/neural_network/CSV/census.csv')
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:
def convert_class(mark):
  if mark == ' >50K':
      return 1
  else:
      return 0

In [0]:
base.income = base.income.apply(convert_class)

In [59]:
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [0]:
X = base.drop('income', axis=1)
y = base.income

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3 )

In [62]:
X_train.shape

(22792, 14)

In [63]:
X_test.shape

(9769, 14)

In [64]:
base.columns

Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')

In [0]:
workclass = tf.feature_column.categorical_column_with_hash_bucket(key='workclass', hash_bucket_size=100)
education = tf.feature_column.categorical_column_with_hash_bucket(key='education', hash_bucket_size=100)
marital_status = tf.feature_column.categorical_column_with_hash_bucket(key= 'marital-status', hash_bucket_size=100)
occupation = tf.feature_column.categorical_column_with_hash_bucket(key='occupation', hash_bucket_size=100)
relationship = tf.feature_column.categorical_column_with_hash_bucket(key='relationship', hash_bucket_size=100)
race = tf.feature_column.categorical_column_with_hash_bucket(key='race', hash_bucket_size=100)
native_country = tf.feature_column.categorical_column_with_hash_bucket(key='native-country', hash_bucket_size=100)

In [66]:
base.sex.unique()

array([' Male', ' Female'], dtype=object)

In [0]:
sex = tf.feature_column.categorical_column_with_vocabulary_list(key='sex', vocabulary_list=[' Male', ' Female'])

In [68]:
base.age.mean()

38.58164675532078

In [69]:
base.age.std()

13.640432553581146

In [0]:
def standardize_age(val):
  return tf.divide(tf.subtract(tf.cast(val, tf.float32), tf.constant(38.58)), tf.constant(13.64))

def standardize_finalweight(val):
  return tf.divide(tf.subtract(tf.cast(val, tf.float32), tf.constant(189778.36)), tf.constant(105549.977))

def standardize_education(val):
  return tf.divide(tf.subtract(tf.cast(val, tf.float32), tf.constant(10.08)), tf.constant(2.57))

def standardize_capitalgain(val):
  return tf.divide(tf.subtract(tf.cast(val, tf.float32), tf.constant(1077.64)), tf.constant(7385.29))

def standardize_capitalloss(val):
  return tf.divide(tf.subtract(tf.cast(val, tf.float32), tf.constant(87.30)), tf.constant(402.96))

def standardize_hour(val):
  return tf.divide(tf.subtract(tf.cast(val, tf.float32), tf.constant(40.43)), tf.constant(12.34))

In [0]:
age = tf.feature_column.numeric_column(key='age', normalizer_fn=standardize_age)
final_weight = tf.feature_column.numeric_column(key='final-weight', normalizer_fn=standardize_finalweight)
education_num = tf.feature_column.numeric_column(key='education-num', normalizer_fn=standardize_education)
capital_gain = tf.feature_column.numeric_column(key='capital-gain', normalizer_fn=standardize_capitalgain)
capital_loos = tf.feature_column.numeric_column(key='capital-loos', normalizer_fn=standardize_capitalloss)
hour_per_week = tf.feature_column.numeric_column(key='hour-per-week', normalizer_fn=standardize_hour)

In [0]:
#columns = [age, workclass, final_weight, education, education_num,
#       marital_status, occupation, relationship, race, sex,
#       capital_gain, capital_loos, hour_per_week, native_country]

In [0]:
#columns

In [0]:
#function_train = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size=21, num_epochs=None, shuffle=True)
#classifier = tf.estimator.DNNClassifier(hidden_units=[8, 8], feature_columns=columns, n_classes=2)
#classifier.train(input_fn=function_train)

In [0]:
embedded_workclass = tf.feature_column.embedding_column(workclass, dimension=len(base.workclass.unique()))
embedded_education = tf.feature_column.embedding_column(education, dimension=len(base.education.unique()))
embedded_marital = tf.feature_column.embedding_column(marital_status, dimension=len(base['marital-status'].unique()))
embedded_occupation = tf.feature_column.embedding_column(occupation, dimension=len(base.occupation.unique()))
embedded_relationship = tf.feature_column.embedding_column(relationship, dimension=len(base.relationship.unique()))
embedded_race = tf.feature_column.embedding_column(race, dimension=len(base.race.unique()))
embedded_sex = tf.feature_column.embedding_column(sex, dimension=len(base.sex.unique()))
embedded_native_country = tf.feature_column.embedding_column(native_country, dimension=len(base['native-country'].unique()))

In [0]:
columns_rna = [age, embedded_workclass, final_weight, embedded_education, education_num,
       embedded_marital, embedded_occupation, embedded_relationship, embedded_race, embedded_sex,
       capital_gain, capital_loos, hour_per_week, embedded_native_country]

In [77]:
function_train = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size=21, num_epochs=None, shuffle=True)
classifier = tf.estimator.DNNClassifier(hidden_units=[8, 8], feature_columns=columns_rna, n_classes=2)
classifier.train(input_fn=function_train, steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmph0mfh82h', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [0]:
function_test = tf.estimator.inputs.pandas_input_fn(x = X_test, y = y_test, batch_size=32, num_epochs=1, shuffle=False )

In [79]:
classifier.evaluate(input_fn=function_test)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-12T22:14:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmph0mfh82h/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.59229s
INFO:tensorflow:Finished evaluation at 2020-06-12-22:14:10
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.85720134, accuracy_baseline = 0.76343536, auc = 0.9101506, auc_precision_recall = 0.77699757, average_loss = 0.30964717, global_step = 10000, label/mean = 0.23656464, loss = 9.885435, precision = 0.7491839, prediction/mean = 0.2311474, recall = 0.59584594
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /tmp/tmph0mfh82h/model.ckpt-10000


{'accuracy': 0.85720134,
 'accuracy_baseline': 0.76343536,
 'auc': 0.9101506,
 'auc_precision_recall': 0.77699757,
 'average_loss': 0.30964717,
 'global_step': 10000,
 'label/mean': 0.23656464,
 'loss': 9.885435,
 'precision': 0.7491839,
 'prediction/mean': 0.2311474,
 'recall': 0.59584594}